# 中文数据处理

本python框架目的在于利用python3 对中文数据进行相关处理工作，以此能够较为稳定的处理字符编码与转换。从而方便提取有用信息。方便进行数据分析。

本文档主要包是进行数据整合工作！

数据 txt文件需要放在同一目录的url文件夹下面

In [ ]:
# encoding=utf-8
import pandas as pd
import numpy as np
import sys 
import os
import glob

# import scipy as sp

## 数据整合


* 把多个数据文件合并成一个
* 分类提取相关类型的土地数据：工业用地
* 数据存储

In [ ]:
# load the file name txt format :
path = ".\\info\\"
# file_list =os.listdir(path)
file_list = glob.glob(path+'*.txt')
file_list
savefile=[]
for element in file_list:
    savefile.append(os.path.splitext(os.path.basename(element)))
savefile


In [ ]:
## 测试

data_fm=pd.read_table(path+'2014-10-4export.txt',sep='delimiter',header=None,encoding='utf-8')
data_fm=data_fm[0].str.split('\t').apply(pd.Series, 1)

## 设置表头，重新排版
data_fm.iloc[0]
data_fm.columns=data_fm.iloc[0]
data_fm=data_fm.reindex(data_fm.index.drop(0))
data_fm=data_fm.drop(data_fm.columns[20:], axis=1)

data_fm

In [ ]:
count=0
for file_ele in file_list:
    
    print(file_ele)
    temp_data_fm = pd.read_table(file_ele,sep='delimiter',header=None,encoding='utf-8')
#     temp_data_fm = pd.read_table(file_ele,sep='delimiter',header=None,encoding='utf-8-sig')
    temp_data_fm=temp_data_fm[0].str.split('\t').apply(pd.Series, 1)
    # delete the na column 
    print(temp_data_fm.shape)
    
    temp_data_fm=temp_data_fm.drop(temp_data_fm.columns[20:], axis=1)
    
    # make column name 
    temp_data_fm.iloc[0]
    temp_data_fm.columns=temp_data_fm.iloc[0]
    temp_data_fm=temp_data_fm.reindex(temp_data_fm.index.drop(0))
    
    ######
    # merge data
    ######
    if count==0:
        data_fm=temp_data_fm
    else:
        data_fm=data_fm.append(temp_data_fm,ignore_index=True)
    
    count+=1


In [ ]:
##-------------------
## 删除重复数据
##-------------------

print(data_fm.shape)
data_fm = data_fm.groupby(level=0).first()
print(data_fm.shape)
print(data_fm[~data_fm['电子监管号'].apply(np.isreal)].shape)

## 分类土地类型

In [ ]:
##-------------------
## 得到工业土地数据 
## 工业用地：拍卖、招标、挂牌
## 仓储用地：拍卖、挂牌
##-------------------

## 判断仓储用地的供地方式类型
data_fm[data_fm['土地用途'].str.contains('仓储用地')==True]['供地方式'].unique()
## 判断土地用途类型
data_fm['土地用途'].unique()    
    
g_data_fm=data_fm.groupby(['土地用途','供地方式'])

# g_data_fm.get_group(('工业用地','拍卖出让'))
fm_industry=pd.DataFrame(columns=data_fm.columns)
supply_way=['拍卖出让', # not so many
            '招标出让', # little
            '挂牌出让'] # large

for ele in supply_way:
    fm_industry=fm_industry.append(g_data_fm.get_group((u'工业用地',ele)),ignore_index=True)

fm_industry=fm_industry.append(g_data_fm.get_group((u'仓储用地','拍卖出让')),ignore_index=True)
fm_industry=fm_industry.append(g_data_fm.get_group((u'仓储用地','挂牌出让')),ignore_index=True)

## 存储数据

In [ ]:
## 存储一级市场数据 
data_fm.to_excel(path+'data_fm.xlsx','Sheet1')
fm_industry.columns
fm_industry.to_excel(path+'data_fm.xlsx','Sheet1')

In [ ]:
fm_industry.columns

In [ ]:
data_fm['土地用途'].unique()

In [ ]:
data_fm[data_fm['土地用途'].str.contains('仓储用地')==True]['供地方式'].unique()